In [1]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig,Model

import sys

In [2]:
ws = Workspace.from_config()

In [3]:
webservice_name = 'churn-classifier'

run = [x for x in ws.experiments['churn-classifier'].get_runs()][0]
run_metrics = run.get_metrics()

print(run)

model = run.register_model(model_name = "churn-classifier", model_path = "outputs/model.pkl", tags={"accuracy":run_metrics["accuracy"],"python version": sys.version[0:6]})

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,auth_enabled=True,enable_app_insights=True)

Run(Experiment: churn-classifier,
Id: 3c5d7741-4579-4f3d-ba70-4e5a4a6738d4,
Type: None,
Status: Completed)


In [4]:
myenv = Environment.get(ws,name="sklearn_20_2")
myenv.inferencing_stack_version = "latest"


inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
                               
try:
    service = Webservice(ws, webservice_name)        
    service.delete()    
except Exception as e:
    print("No Existing Service")

service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

#service.get_logs()
service.wait_for_deployment(show_output = True)

#print(service.get_keys())
print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-15 15:13:07+00:00 Creating Container Registry if not exists.
2021-10-15 15:13:07+00:00 Registering the environment.
2021-10-15 15:13:07+00:00 Use the existing image.
2021-10-15 15:13:08+00:00 Generating deployment configuration.
2021-10-15 15:13:08+00:00 Submitting deployment to compute.
2021-10-15 15:13:12+00:00 Checking the status of deployment churn-classifier..
2021-10-15 15:20:41+00:00 Checking the status of inference endpoint churn-classifier.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 48e16d1c-0d46-478c-acca-9087a936fd43
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: churn-classifier. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image aiwregistry.azurecr.io/azureml/azureml_e7606bffe4681d4b0c5a18a2f4070c1e locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more informa

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 48e16d1c-0d46-478c-acca-9087a936fd43
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: churn-classifier. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image aiwregistry.azurecr.io/azureml/azureml_e7606bffe4681d4b0c5a18a2f4070c1e locally. Please refer to https://aka

In [1]:
import sklearn 
print (sklearn.__version__)

0.22.2.post1
